<a href="https://www.nvidia.com/dli"> <img src="images/DLI Header.png" alt="Header" style="width: 400px;"/> </a>

# CUDA Python與Numba中多維度Grid與Shared Memory

在此段落中，我們要再介紹一些在CUDA中使用Numba程式設計的技巧。首先說明CUDA如何提供建立2維和3維Block及Grid的能力，藉此促進處理2維和3維資料時的程式設計工作。接著要介紹由程式設計師管理的記憶體空間，稱為**共享記憶體(Shared Memory)**，可用於同一Block中Thread之間的快速溝通。您可利用這些技巧最佳化2維矩陣乘法Kernel。

本段附錄中也透過範例說明如何減少矩陣轉置演算法的**共享記憶體 Bank conflicts**，並提供參考資訊連結。

## 目標

完成本段必修部分後，您將能夠：

* 利用多維度Block及Grid在多維度資料集上進行GPU加速平行作業。
* 利用共享記憶體將資料暫存於晶片並減少全域記憶體存取。

## 2維及3維Block與Grid

Grid和Block皆能配置成分別包含Block或Thread的2維或3維集合。對於經常使用2維或3維資料集的程式設計師來說，這主要是為了方便。以下是一個簡單的語法範例。您可能需要先閱讀Kernel定義及其啟動，才能明白這個概念。

In [1]:
import numpy as np
from numba import cuda

In [2]:
A = np.zeros(16).reshape(4, 4).astype(np.int32)
d_A = cuda.to_device(A)

@cuda.jit
def add_2D_coordinates(A):
    # By passing `2`, we get the thread's unique x and y coordinates in the 2D grid
    x, y = cuda.grid(2)
    
    A[y][x] = x + y

In [3]:
# Here we create a 2D grid with 4 blocks in a 2x2 structure, each with 4 threads in a 2x2 structure
# by using a Python tuple to signify grid and block dimensions.
blocks = (2, 2)
threads_per_block = (2, 2)

add_2D_coordinates[blocks, threads_per_block](d_A)
print(d_A.copy_to_host())

[[0 1 2 3]
 [1 2 3 4]
 [2 3 4 5]
 [3 4 5 6]]


### 關於練習的附加資訊

請切記，GPU必須有足夠支援SM的Grid大小，在處理作業並等待延遲終止的同時執行其他工作，才能夠展現良好性能。雖然如此，但本段中的幾項練習卻是要請您撰寫出不遵循這種最佳實務的Kernel。這是幫助您聚焦於語法和一些處理多重維度的基本模式，屬於介紹性質，較不具有實際效益。

### 練習：在GPU上增設2D矩陣

在此練習中，您要修改Kernel定義及其執行配置以平行執行2D矩陣加法。首先請撰寫一個只有在連同維度與資料匹配的Grid一起啟動時才能作用的Kernel。若有問題請參考[解決方案](../../../../edit/tasks/task3/task/solutions/add_matrix_solution.py)。

In [4]:
# This function is written to add two 1D vectors. Refactor it to add 2D matrices
"""
@cuda.jit
def add_matrix(A, B, C):
    i = cuda.grid(1)
    
    C[i] = A[i] + B[i]
"""

@cuda.jit
def add_matrix(A, B, C):
    i,j = cuda.grid(2)
    
    C[j,i] = A[j,i] + B[j,i]

In [5]:
# Do not modify the values in this cell, which defines 2D matrices of size 36*36
A = np.arange(36*36).reshape(36, 36).astype(np.int32)
B = A * 2
C = np.zeros_like(A)
d_A = cuda.to_device(A)
d_B = cuda.to_device(B)
d_C = cuda.to_device(C)

In [6]:
# Refactor the launch configuration to use a 2D grid with 2D blocks
blocks = 36
threads_per_block = 36

# This launch will throw a Typing error until refactor the definition above to operate on 2D arrays
add_matrix[blocks, threads_per_block](d_A, d_B, d_C)

In [7]:
from numpy import testing
output = d_C.copy_to_host()
solution = A+B
# This assertion will fail unles the output and solution are equal
testing.assert_array_equal(output, solution)

### 練習：在GPU 上實作2D矩陣乘法

在這項練習中，您要針對一個為2D[矩陣乘法](https://en.wikipedia.org/wiki/Matrix_multiplication)運算計算元素的邏輯製作其Kernel。就向您剛才撰寫的矩陣相加Kernel一樣，這也是個簡單的Kernel，需要與傳入矩陣的Grid維度相符。若有問題請參考[解決方案](../../../../edit/tasks/task3/task/solutions/matrix_multiply_solution.py)。

In [8]:
import numpy as np
from numba import cuda
"""

@cuda.jit
def mm(a, b, c):
    row, column = cuda.grid(2)
    sum = 0
    
    ###
    # TODO: Build the rest of this kernel to calculate the value for one element in the output matrix.
    ###
        
    c[row][column] = sum
"""    
    
from numba import cuda

@cuda.jit
def mm(a, b, c):
    column, row = cuda.grid(2)
    sum = 0
    
    for i in range(a.shape[0]):
        sum += a[row][i] * b[i][column]
        
    c[row][column] = sum

In [9]:
# Do not modify the values in this cell
a = np.arange(16).reshape(4,4).astype(np.int32)
b = np.arange(16).reshape(4,4).astype(np.int32)
c = np.zeros_like(a)

d_a = cuda.to_device(a)
d_b = cuda.to_device(b)
d_c = cuda.to_device(c)

grid = (2,2)
block = (2,2)
mm[grid, block](d_a, d_b, d_c)

In [10]:
from numpy import testing
solution = a@b
output = d_c.copy_to_host()
# This assertion will fail until you successfully implement the kernel
testing.assert_array_equal(output, solution)

## 多重維度的跨度迴圈

正如同我們可利用Numba的`cuda.gridsize(1)`取得一個Grid中的Thread總數，我們也可以利用`cuda.gridsize(2)`得知2D Grid中每一方向Thread總數的變數。舉例而言，當2D資料集大於Grid，而我們希望每一個Thread都能跨越迴圈中的Grid而順利執行所有必要作業時，這種方法就極為實用。

就像1D Grid跨步迴圈的情形一樣，這項技巧也能讓我們靈活設定Grid和Block的大小，不受資料形狀影響。以下範例說明2D Grid跨度迴圈的使用，最終列印出有關Grid中哪些Thread作用於矩陣中哪些元素的資訊。

In [11]:
import numpy as np
from numba import cuda
@cuda.jit
def add_2D_coordinates_stride(A):

    grid_y, grid_x = cuda.grid(2)
    # By passing `2`, we get the grid size in both the x an y dimensions
    stride_y, stride_x = cuda.gridsize(2)
    
    for data_i in range(grid_x, A.shape[0], stride_x):
        for data_j in range(grid_y, A.shape[1], stride_y):
            A[data_i][data_j] = grid_x + grid_y

現在我們建立一個在3x2 結構中具有6個Block的2D Grid，每個Block中也是由6個Thread構成的3x2結構。這個Grid小於我們的總資料集，且形狀無法平均配適資料集的維度。這個Kernel還是能夠存取資料中的每一元素。執行此單元後，嘗試隨著Grid形狀改動不同資料形狀。執行程式碼之前，先嘗試預測輸出矩陣的數值。

In [12]:
A = np.zeros(55).reshape(11, 5).astype(np.int32)
d_A = cuda.to_device(A)

blocks = (3, 2)
threads_per_block = (3, 2)

# With this configuration, `stride_x` will be 9, and `stride_y` will be 4
add_2D_coordinates_stride[blocks, threads_per_block](d_A)
print(d_A.copy_to_host())

[[0 1 2 3 4]
 [1 2 3 4 5]
 [2 3 4 5 6]
 [3 4 5 6 7]
 [0 1 2 3 4]
 [1 2 3 4 5]
 [2 3 4 5 6]
 [3 4 5 6 7]
 [0 1 2 3 4]
 [1 2 3 4 5]
 [2 3 4 5 6]]


### 練習：處理大於Grid尺寸的2D矩陣加法

在此練習中，您要將簡單的矩陣加法Kernel修改為至少能夠處理任意大小資料集的尺寸。若有問題請參考[解決方案](../../../../edit/tasks/task3/task/solutions/add_matrix_stride_solution.py)。

In [13]:
# Currently this kernel will only work correctly when passed matrices that are of the same size as the grid.
# Refactor using a strid in 2D so that it can work on data sets of an arbitrary size.
"""
@cuda.jit
def add_matrix_stride(A, B, C):
    j,i = cuda.grid(2)
    
    C[i,j] = A[i,j] + B[i,j]
"""   
    
@cuda.jit
def add_matrix_stride(A, B, C):

    y, x = cuda.grid(2)
    stride_y, stride_x = cuda.gridsize(2)
    
    for i in range(x, A.shape[0], stride_x):
        for j in range(y, A.shape[1], stride_y):
            C[i][j] = A[i][j] + B[i][j]

In [14]:
# Please don't modify the values in this cell. They create a scenario where the data is
# larger than the grid size.
A = np.arange(64*64).reshape(64, 64).astype(np.int32)
B = A * 2
C = np.zeros_like(A)
d_A = cuda.to_device(A)
d_B = cuda.to_device(B)
d_C = cuda.to_device(C)

blocks = (6,6)
threads_per_block = (6,6)

add_matrix_stride[blocks, threads_per_block](d_A, d_B, d_C)

In [15]:
from numpy import testing
output = d_C.copy_to_host()
solution = A+B
# This assertion will fail unles the output and solution are equal
testing.assert_array_equal(output, solution)

### 練習：處理大於Grid尺寸的2D矩陣乘法

在此練習中，您要完成一個能夠處理任意Grid和資料集形狀的矩陣乘法Kernel。您只需要處理包含`TODO`的兩行，利用`grid_`和`stride_`值來將執行Kernel的工作正確映射至資料中。若有問題請參考[解決方案](../../../../edit/tasks/task3/task/solutions/matrix_multiply_stride_solution.py)。

In [19]:
import numpy as np
from numba import cuda
"""
@cuda.jit
def mm_stride(A, B, C):

    grid_row, grid_column = cuda.grid(2)
    stride_row, stride_column = cuda.gridsize(2)
    
    for data_row in range(0): # TODO: replace 0 with values that will correctly set data_row
        for data_column in range(0): # TODO: replace 0 with values that will correctly set data_column
            sum = 0
            for i in range(A.shape[1]): # B.shape[0] would also be okay here
                sum += A[data_row][i] * B[i][data_column]
                
            C[data_row][data_column] = sum
"""

@cuda.jit
def mm_stride(A, B, C):

    grid_column, grid_row = cuda.grid(2)
    stride_column, stride_row = cuda.gridsize(2)
    
    for data_row in range(grid_row, A.shape[0], stride_row):
        for data_column in range(grid_column, B.shape[1], stride_column):
            sum = 0
            for i in range(A.shape[1]): # `range(B.shape[0])` is also okay
                sum += A[data_row][i] * B[i][data_column]
                
            C[data_row][data_column] = sum
            
n = 1024
a = np.arange(n*n).reshape(n,n).astype(np.int32)
b = np.arange(n*n).reshape(n,n).astype(np.int32)
c = np.zeros((a.shape[0], b.shape[1])).astype(np.int32)

d_a = cuda.to_device(a)
d_b = cuda.to_device(b)
d_c = cuda.to_device(c)

ts = (32,32)
bs = (32,32)

mm_stride[bs, ts](d_a, d_b, d_c)

In [20]:
# Please do not modify this cell. The strange dimensions of this data, and
# the grid below are being set to make sure your kernel correctly handles arbitrary
# data and grid sizes.

a = np.arange(12).reshape(3,4).astype(np.int32)
b = np.arange(24).reshape(4,6).astype(np.int32)
c = np.zeros((a.shape[0], b.shape[1])).astype(np.int32)

d_a = cuda.to_device(a)
d_b = cuda.to_device(b)
d_c = cuda.to_device(c)

ts = (4, 3)
bs = (3, 7)

mm_stride[bs, ts](d_a, d_b, d_c)

In [21]:
from numpy import testing
solution = a@b
output = d_c.copy_to_host()
# This assertion will fail until you correctly update the kernel above.
testing.assert_array_equal(output, solution)

## 共享記憶體

至今我們已將主機與裝置記憶體區分開來，但只把裝置記憶體當成單一種類的記憶體。其實CUDA分為更細的[記憶體階層](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#memory-hierarchy)。我們至今所用過的裝置記憶體稱為**全域記憶體**，可為裝置上的任何Thread或Block所用，可隨著應用程式使用時間存在，記憶體空間相對較大。

最後一個要討論的主題是如何利用On Chip memory上這個稱為**共享記憶體(Shared memory)**的區域。共享記憶體是一種由程式設計師定義的快取，其大小有限，[取決於所用GPU](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#compute-capabilities)，且是由同一Block中的所有Thread**共用**。這是一項稀少的資源，無法由其所在Block以外的Thread所存取，且在Kernel完成執行後不會保留。但共享記憶體的頻寬遠大於全域記憶體，因此可於許多Kernel達成良好效用，特別是最佳化性能。

以下是共享記憶體的常見用途：

 * 對讀取自全域記憶體且需要在Block內多次讀取的記憶體進行快取。
 * 緩衝來自Thread的輸出，使其可於寫回全域記憶體前先行合併。
 * 暫存Block內分scatter/gather操作的資料。

### 共享記憶體語法

Numba提供的[函數](https://numba.pydata.org/numba-doc/dev/cuda/memory.html#shared-memory-and-thread-synchronization)用於分配共享記憶體以及達成同一Block中Thread之間的同步，後者通常為對共享記憶體讀寫平行Thread之後所必須。

宣告共享記憶體時，要提供共享陣列的形狀及其類型，使用[Numba類型](https://numba.pydata.org/numba-doc/dev/reference/types.html#numba-types)。**陣列的形狀必須為常數**，因此無法使用傳送至函數的引數，或是像`numba.cuda.blockDim.x`這種提供的變數，或是`cuda.griddim`的值。這個範例顯示的語法中以註解指出從主機記憶體到全域裝置記憶體，到共享記憶體，再回到全域裝置記憶體，最終回到主機記憶體的移動：

In [22]:
import numpy as np
from numba import types, cuda

@cuda.jit
def swap_with_shared(x, y):
    # Allocate a 4 element vector containing int32 values in shared memory.
    temp = cuda.shared.array(4, dtype=types.int32)
    
    idx = cuda.grid(1)
    
    # Move an element from global memory into shared memory
    temp[idx] = x[idx]
    
    # cuda.syncthreads will force all threads in the block to synchronize here, which is necessary because...
    cuda.syncthreads()
    #...the following operation is reading an element written to shared memory by another thread.
    
    # Move an element from shared memory back into global memory
    y[idx] = temp[cuda.blockDim.x - cuda.threadIdx.x - 1] # swap elements

In [23]:
x = np.arange(4).astype(np.int32)
y = np.zeros_like(x)

# Move host memory to device (global) memory
d_x = cuda.to_device(x)
d_y = cuda.to_device(y)

swap_with_shared[1, 4](d_x, d_y)

In [24]:
# Move device (global) memory back to the host
d_y.copy_to_host()

array([3, 2, 1, 0], dtype=int32)

### 共享記憶體範例：矩陣轉置

此範例一方面說明共享記憶體的功用，另一方面再次說明2維CUDA程式設計，現在我們要撰寫的矩陣轉置Kernel是將以 row-major的2D陣列改為column-major 。(這是基於Mark Harris的[高效率矩陣轉置](https://devblogs.nvidia.com/parallelforall/efficient-matrix-transpose-cuda-cc/)部落格貼文)。

首先進行簡單的方案，讓每一個Thread僅使用全域記憶體獨立讀寫個別元素。

In [25]:
TILE_DIM = 32
BLOCK_ROWS = 8

@cuda.jit
def transpose(a_in, a_out):
    y = cuda.blockIdx.x * TILE_DIM + cuda.threadIdx.x
    x = cuda.blockIdx.y * TILE_DIM + cuda.threadIdx.y

    for j in range(0, TILE_DIM, BLOCK_ROWS):
        a_out[x + j, y] = a_in[y, x + j]

In [26]:
size = 8192
a_in = cuda.to_device(np.arange(size*size, dtype=np.int32).reshape((size, size)))
a_out = cuda.device_array_like(a_in)

print(a_in.copy_to_host())

[[       0        1        2 ...     8189     8190     8191]
 [    8192     8193     8194 ...    16381    16382    16383]
 [   16384    16385    16386 ...    24573    24574    24575]
 ...
 [67084288 67084289 67084290 ... 67092477 67092478 67092479]
 [67092480 67092481 67092482 ... 67100669 67100670 67100671]
 [67100672 67100673 67100674 ... 67108861 67108862 67108863]]


In [27]:
grid_shape = (int(size/TILE_DIM), int(size/TILE_DIM))
%timeit transpose[grid_shape,(TILE_DIM, BLOCK_ROWS)](a_in, a_out); cuda.synchronize()
print(a_out.copy_to_host())

3.99 ms ± 4.11 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
[[       0     8192    16384 ... 67084288 67092480 67100672]
 [       1     8193    16385 ... 67084289 67092481 67100673]
 [       2     8194    16386 ... 67084290 67092482 67100674]
 ...
 [    8189    16381    24573 ... 67092477 67100669 67108861]
 [    8190    16382    24574 ... 67092478 67100670 67108862]
 [    8191    16383    24575 ... 67092479 67100671 67108863]]


現在我們使用共享記憶體將32x32區塊複製一次。 我們要用區塊大小的全域值，使其於編譯時為已知。

In [28]:
import numba.types

@cuda.jit
def tile_transpose(a_in, a_out):
    # THIS CODE ASSUMES IT IS RUNNING WITH A BLOCK DIMENSION OF (TILE_SIZE x TILE_SIZE)
    # AND INPUT IS A MULTIPLE OF TILE_SIZE DIMENSIONS
    tile = cuda.shared.array((TILE_DIM, TILE_DIM), numba.types.int32)

    x = cuda.blockIdx.x * TILE_DIM + cuda.threadIdx.x
    y = cuda.blockIdx.y * TILE_DIM + cuda.threadIdx.y
    
    for j in range(0, TILE_DIM, BLOCK_ROWS):
        tile[cuda.threadIdx.y + j, cuda.threadIdx.x] = a_in[y + j, x] # move tile into shared memory

    cuda.syncthreads()  # wait for all threads in the block to finish updating shared memory

    # Compute transposed offsets
    x = cuda.blockIdx.y * TILE_DIM + cuda.threadIdx.x
    y = cuda.blockIdx.x * TILE_DIM + cuda.threadIdx.y

    for j in range(0, TILE_DIM, BLOCK_ROWS):
        a_out[y + j, x] = tile[cuda.threadIdx.x, cuda.threadIdx.y + j];


In [29]:
a_out = cuda.device_array_like(a_in)

%timeit tile_transpose[grid_shape,(TILE_DIM, BLOCK_ROWS)](a_in, a_out); cuda.synchronize()
print(a_out.copy_to_host())

3.24 ms ± 38.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
[[       0     8192    16384 ... 67084288 67092480 67100672]
 [       1     8193    16385 ... 67084289 67092481 67100673]
 [       2     8194    16386 ... 67084290 67092482 67100674]
 ...
 [    8189    16381    24573 ... 67092477 67100669 67108861]
 [    8190    16382    24574 ... 67092478 67100670 67108862]
 [    8191    16383    24575 ... 67092479 67100671 67108863]]


除了已加速的代碼之外，這已經是一個非常好的加速效果。

## 評估

以下練習要請您充分運用至今所學。此處與先前練習的差異在於，沒有解決方案程式碼供您使用，並且您必須執行幾個額外的步驟以「執行評估」並取得您作答的分數。**開始處理前請仔細閱讀指示，以便順利完成評估。**

### 如何進行評估

經執行以下步驟，完成評估：

1. 遵循以下指示，按照您一般進行練習的方式處理下方單元。
2. 達到您滿意的程度後，請遵循以下指示，複製程式碼並將之貼到連結的原始碼檔案中。貼入程式碼後請務必記得儲存檔案。
3. 返回您用於啟動此本筆記的瀏覽器分頁，然後點擊**「評估(Assess)」**鈕。數秒後會顯示分數以及實用訊息。

歡迎隨時點擊**評估**鈕，若您首次未能通過，您可額外修改程式碼並重複執行步驟1至3。祝您好運！

### 使用共享記憶體進行矩陣乘法

在此練習中，您所要完成的矩陣乘法Kernel可利用共享記憶體暫存來自輸入矩陣的數值，因此只需要被全域記憶體存取一次，之後Thread輸出元素的計算可利用快取值。此項評估旨在測試您能否推理2D平行問題並利用共享記憶體。這個問題的算術密度不大，並且我們不打算使用龐大資料集，因此相對於相當簡單的CPU版本，加速效果可能不大。但您將這些技巧運用於各種涉及2D資料集的程式加速需求中。

為持續聚焦於共享記憶體，這個問題假設MxN和 NxM維度的輸入向量具有每一Block有NxN個Thread，且每一Grid有M/N個Block。這表示共享記憶體暫存與每一Block中Thread數量相等的元素，足以提供計算所需的所有輸入矩陣元素，且不需要Grid跨步。

下圖顯示輸入矩陣、輸出矩陣、Block將計算其數值的輸出矩陣區域、Block將暫存的輸入矩陣區域以及該Block中單一Thread的輸出元素和輸入元素：

![matrix multiply diagram](images/mm_image.png)

共享記憶體塊取已經分配到Kernel中，您的任務有兩方面：
1. 使用Block中的每一個Thread，在各快取中放入一個元素。
2. 使用共享記憶體快取計算每一個Thread的`sum`值。

務必執行可能需要的Thread同步，以免其他Thread寫入的尚未可用的快取值。

In [30]:
# Leave the values in this cell alone
M = 128
N = 32

# Input vectors of MxN and NxM dimensions
a = np.arange(M*N).reshape(M,N).astype(np.int32)
b = np.arange(M*N).reshape(N,M).astype(np.int32)
c = np.zeros((M, M)).astype(np.int32)

d_a = cuda.to_device(a)
d_b = cuda.to_device(b)
d_c = cuda.to_device(c)

# NxN threads per block, in 2 dimensions
block_size = (N,N)
# MxM/NxN blocks per grid, in 2 dimensions
grid_size = (int(M/N),int(M/N))

在修改以下單元的`mm_shared`之後，要執行評估前，請將此單元的內容貼入[**`assessment/definition.py`**](../../../../edit/tasks/task3/task/assessment/definition.py)

In [38]:
import numpy as np
from numba import cuda, types
"""
@cuda.jit
def mm_shared(a, b, c):
    column, row = cuda.grid(2)
    sum = 0

    # `a_cache` and `b_cache` are already correctly defined
    a_cache = cuda.shared.array(block_size, types.int32)
    b_cache = cuda.shared.array(block_size, types.int32)

    # TODO: use each thread to populate one element each a_cache and b_cache
    
    for i in range(a.shape[1]):
        # TODO: calculate the `sum` value correctly using values from the cache 
        sum += a_cache[0][0] * b_cache[0][0]
        
    c[row][column] = sum
"""
    
@cuda.jit
def mm_shared(a, b, c):
    sum = 0

    # `a_cache` and `b_cache` are already correctly defined
    a_cache = cuda.shared.array(block_size, types.int32)
    b_cache = cuda.shared.array(block_size, types.int32)

    # TODO: use each thread to populate one element each a_cache and b_cache
    x,y = cuda.grid(2)
    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    bpg = cuda.gridDim.x
    TPB = int(N)
    
    for i in range(a.shape[1] / TPB):
        a_cache[tx, ty] = a[x, ty + i * TPB]
        b_cache[tx, ty] = b[tx + i * TPB, y]
    
    cuda.syncthreads()
    for j in range(TPB):#a.shape[1]):
        # TODO: calculate the `sum` value correctly using values from the cache 
        sum += a_cache[tx][j] * b_cache[j][ty]
    cuda.syncthreads()    
    c[x][y] = sum

In [39]:
# There's no need to update this kernel launch
mm_shared[grid_size, block_size](d_a, d_b, d_c)

In [40]:
# Do not modify the contents in this cell
from numpy import testing
solution = a@b
output = d_c.copy_to_host()
# This assertion will fail until you correctly update the kernel above.
testing.assert_array_equal(output, solution)

## 總結

完成本段後您能夠：

* 利用多維度Block及Grid在多維度資料集上進行GPU加速平行作業。
* 利用共享記憶體將資料暫存於晶片並減少較慢的全域記憶體存取。

## 下載內容

如需下載本篇筆記內容，請執行以下單元，然後點擊下方的下載連結。請注意：若您是在本機Jupyter伺服器中執行此notebeook，notebook中的部分檔案路徑連結可能會因配合我們本身平台的資料夾結構而失效。您仍可透過Jupyter檔案瀏覽器查看檔案。

In [34]:
!tar -zcvf section3.tar.gz .

./
./.ipynb_checkpoints/
./.ipynb_checkpoints/Multidimensional Grids and Shared Memory for CUDA Python with Numba-checkpoint.ipynb
./solutions/
./solutions/add_matrix_stride_solution.py
./solutions/add_matrix_solution.py
./solutions/matrix_multiply_solution.py
./solutions/matrix_multiply_stride_solution.py
./solutions/monte_carlo_pi_solution.py
./images/
./images/DLI Header.png
./images/mm_image.png
./assessment/
./assessment/definition.py
./Multidimensional Grids and Shared Memory for CUDA Python with Numba.ipynb
tar: .: file changed as we read it


[下載本段檔案。](files/section3.tar.gz)

## 附錄：無Bank Conflict的矩陣轉置

共享記憶體是儲存於**Banks**中。共有32個Banks可用於儲存共享記憶體，且不使用相同記憶體Bank的記憶體讀寫可以同時執行。當平行Thread試圖存取相同Bank中的記憶體時，即稱之為**Bank Conflict**，會導致操作序列化。即便發生Bank Conflict，共享記憶體的速度仍然很快，但建立可避免Bank Conflict的記憶體存取方式，就能進一步最佳化應用程式。

我們在此僅使用一個範例，若需詳細資訊，可參考[高效率矩陣轉置](https://devblogs.nvidia.com/parallelforall/efficient-matrix-transpose-cuda-cc/)部落格貼文。

In [35]:
from numba import cuda, types
import numpy as np

TILE_DIM = 32
BLOCK_ROWS = 8
TILE_DIM_PADDED = TILE_DIM + 1  # Read Mark Harris' blog post to find out why this improves performance!
                                # https://devblogs.nvidia.com/parallelforall/efficient-matrix-transpose-cuda-cc/

@cuda.jit
def tile_transpose_no_bank_conflict(a_in, a_out):
    # THIS CODE ASSUMES IT IS RUNNING WITH A BLOCK DIMENSION OF (TILE_SIZE x TILE_SIZE)
    # AND INPUT IS A MULTIPLE OF TILE_SIZE DIMENSIONSx
    tile = cuda.shared.array((TILE_DIM, TILE_DIM_PADDED), types.int32)

    x = cuda.blockIdx.x * TILE_DIM + cuda.threadIdx.x
    y = cuda.blockIdx.y * TILE_DIM + cuda.threadIdx.y
    
    for j in range(0, TILE_DIM, BLOCK_ROWS):
        tile[cuda.threadIdx.y + j, cuda.threadIdx.x] = a_in[y + j, x] # move tile into shared memory

    cuda.syncthreads()  # wait for all threads in the block to finish updating shared memory

    # Compute transposed offsets
    x = cuda.blockIdx.y * TILE_DIM + cuda.threadIdx.x
    y = cuda.blockIdx.x * TILE_DIM + cuda.threadIdx.y

    for j in range(0, TILE_DIM, BLOCK_ROWS):
        a_out[y + j, x] = tile[cuda.threadIdx.x, cuda.threadIdx.y + j];


In [36]:
size = 8192
a_in = cuda.to_device(np.arange(size*size, dtype=np.int32).reshape((size, size)))
a_out = cuda.device_array_like(a_in)

print(a_in.copy_to_host())

[[       0        1        2 ...     8189     8190     8191]
 [    8192     8193     8194 ...    16381    16382    16383]
 [   16384    16385    16386 ...    24573    24574    24575]
 ...
 [67084288 67084289 67084290 ... 67092477 67092478 67092479]
 [67092480 67092481 67092482 ... 67100669 67100670 67100671]
 [67100672 67100673 67100674 ... 67108861 67108862 67108863]]


In [37]:
grid_shape = (int(size/TILE_DIM), int(size/TILE_DIM))

%timeit tile_transpose_no_bank_conflict[grid_shape,(TILE_DIM, BLOCK_ROWS)](a_in, a_out); cuda.synchronize()
print(a_out.copy_to_host())

2.96 ms ± 2.99 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
[[       0     8192    16384 ... 67084288 67092480 67100672]
 [       1     8193    16385 ... 67084289 67092481 67100673]
 [       2     8194    16386 ... 67084290 67092482 67100674]
 ...
 [    8189    16381    24573 ... 67092477 67100669 67108861]
 [    8190    16382    24574 ... 67092478 67100670 67108862]
 [    8191    16383    24575 ... 67092479 67100671 67108863]]


<a href="https://www.nvidia.com/dli"> <img src="images/DLI Header.png" alt="Header" style="width: 400px;"/> </a>